In [ ]:
#factory method

没看完

In [ ]:

#引语

"""
Factory Method is a creational design pattern used to create concrete implementations of a common interface.

It separates the process of creating an object from the code that depends on the interface of the object.

For example, an application requires an object with a specific interface to perform its tasks. The concrete implementation of the interface is identified by some parameter.

Instead of using a complex if/elif/else conditional structure to determine the concrete implementation, the application delegates that decision to a separate component that creates the concrete object. With this approach, the application code is simplified, making it more reusable and easier to maintain.

Imagine an application that needs to convert a Song object into its string representation using a specified format. Converting an object to a different representation is often called serializing. You’ll often see these requirements implemented in a single function or method that contains all the logic and implementation, like in the following code:
"""
#In the example above, you have a basic Song class to represent a song and a SongSerializer class that can convert a song object into its string representation according to the value of the format parameter.
#这个方法定义了两个类，song和songserializer,第二个类中使用了大段的if elif else

# In serializer_demo.py
import json
import xml.etree.ElementTree as et

class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)
#调用
"""
import serializer_demo as sd
>>> song = sd.Song('1', 'Water of Love', 'Dire Straits')
>>> serializer = sd.SongSerializer() #这里先引用，再调用song class创建的song

>>> serializer.serialize(song, 'JSON')
'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

>>> serializer.serialize(song, 'XML')
'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

>>> serializer.serialize(song, 'YAML')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "./serializer_demo.py", line 30, in serialize
    raise ValueError(format)
ValueError: YAML
"""
#但是，上面代码过于复杂了。下面这段总结：每次如果改动song的实例属性，也会要求改变相应songserial的实例属性。
#Complex logical code uses if/elif/else structures to change the behavior of an application. Using if/elif/else conditional structures makes the code harder to read, harder to understand, and harder to maintain.
"""
The .serialize() method in SongSerializer will require changes for many different reasons. 
This increases the risk of introducing new defects or breaking existing functionality when changes are made. 
Let’s take a look at all the situations that will require modifications to the implementation:

    When a new format is introduced: The method will have to change to implement the serialization to that format.

    When the Song object changes: Adding or removing properties to the Song class will require the implementation to change in order to accommodate the new structure.

    When the string representation for a format changes (plain JSON vs JSON API): The .serialize() method will have to change if the desired string representation for a format changes because the representation is hard-coded in the .serialize() method implementation.

"""

In [ ]:
#尝试refactoring上面的代码结构，变得更加稳定易读易于更改。 NB： refactoring不是factory method
"""
refactoring as “the process of changing a software system in such a way that does not alter the external behavior of the code yet improves its internal structure.”

Refactoring Code Into the Desired Interface

The desired interface is an object or a function that takes a Song object and returns a string representation.
"""

class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)
        elif format == 'XML':
            return self._serialize_to_xml(song)
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

    #但更改后仍为两个类，song与songserializer。这里更改了songserilr的结构，把if elif else集成在一起，再下发给下面的两个方法。易读性++
    

In [ ]:
#进一步改善，但还不是factory method
"""
The central idea in Factory Method is to provide a separate component with 
the responsibility to decide which concrete implementation should be used based on some specified parameter. 
That parameter in our example is the format.

To complete the implementation of Factory Method, you add a new method ._get_serializer() that takes the desired format. 
This method evaluates the value of format and returns the matching serialization function:
"""
class SongSerializer:
    def serialize(self, song, format):  #主要引用这个，def了一个product component作为method。我们调用这个client，io一个song，op一个字符表达
        #This is referred to as the client component of the pattern. 
        #The interface defined is referred to as the product component. 
        #In our case, the product is a function that takes a Song and returns a string representation.
        
        serializer = self._get_serializer(format)#先是输入format，转到_get_serializer判断类型，再转到具体字符转换的func，最终return输出
        return serializer(song)

    def _get_serializer(self, format):#选择interface
        if format == 'JSON':
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):#concrete implementation
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

"""
The final implementation shows the different components of Factory Method. 
The .serialize() method is the application code that depends on an interface to complete its task.



The ._serialize_to_json() and ._serialize_to_xml() methods are concrete implementations of the product.
Finally, the ._get_serializer() method is the creator component. The creator decides which concrete implementation to use.

Because you started with some existing code, all the components of Factory Method are members of the same class SongSerializer.


"""
#注意这里没有function用到self，说明可以被单独定义

In [ ]:
#将各个方法定义为单独的func（不属于类）
class SongSerializer:
    def serialize(self, song, format):
        serializer = get_serializer(format)
        return serializer(song)


def get_serializer(format):
    if format == 'JSON':
        return _serialize_to_json
    elif format == 'XML':
        return _serialize_to_xml
    else:
        raise ValueError(format)


def _serialize_to_json(song):
    payload = {
        'id': song.song_id,
        'title': song.title,
        'artist': song.artist
    }
    return json.dumps(payload)


def _serialize_to_xml(song):
    song_element = et.Element('song', attrib={'id': song.song_id})
    title = et.SubElement(song_element, 'title')
    title.text = song.title
    artist = et.SubElement(song_element, 'artist')
    artist.text = song.artist
    return et.tostring(song_element, encoding='unicode')
#调用
"""
>>> import serializer_demo as sd
>>> song = sd.Song('1', 'Water of Love', 'Dire Straits')
>>> serializer = sd.SongSerializer()

>>> serializer.serialize(song, 'JSON')    与block1调用完全相同，结果完全相同
'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

>>> serializer.serialize(song, 'XML')
'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

>>> serializer.serialize(song, 'YAML')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "./serializer_demo.py", line 13, in serialize
    serializer = get_serializer(format)
  File "./serializer_demo.py", line 23, in get_serializer
    raise ValueError(format)
ValueError: YAML

"""

In [ ]:
#factory method
"""
优点
Replacing complex logical code:
Constructing related objects from external data:
Supporting multiple implementations of the same feature: 
Combining similar features under a common interface: 
Integrating related external services: 
"""




#An Object Serialization Example
#总结下面这段：之前代码的需求是想要serialize song objects，实际上想要拓展可识别模式，我们可以define an interface Serializer that can have multiple implementations, one per format.
"""
The basic requirements for the example above are that you want to serialize Song objects into their string representation. 
It seems the application provides features related to music, so it is plausible that the application will need to serialize other type of objects like Playlist or Album.

Ideally, the design should support adding serialization for new objects by implementing new classes without requiring changes to the existing implementation. 
The application requires objects to be serialized to multiple formats like JSON and XML, so it seems natural to define an interface Serializer that can have multiple implementations, one per format.

The interface implementation might look something like this:
"""
#interface implementation
# In serializers.py

import json
import xml.etree.ElementTree as et 

class JsonSerializer:
    def __init__(self):
        self._current_object = None

    def start_object(self, object_name, object_id):
        self._current_object = {
            'id': object_id
        }

    def add_property(self, name, value):
        self._current_object[name] = value

    def to_str(self):
        return json.dumps(self._current_object)


class XmlSerializer:
    def __init__(self):
        self._element = None

    def start_object(self, object_name, object_id):
        self._element = et.Element(object_name, attrib={'id': object_id})

    def add_property(self, name, value):
        prop = et.SubElement(self._element, name)
        prop.text = value

    def to_str(self):
        return et.tostring(self._element, encoding='unicode')
#这里，分别定义了两个类的操作
"""
The Serializer interface is an abstract concept due to the dynamic nature of the Python language. 
Static languages like Java or C# require that interfaces be explicitly defined. 
In Python, any object that provides the desired methods or functions is said to implement the interface. 
The example defines the Serializer interface to be an object that implements the following methods or functions:

    .start_object(object_name, object_id)
    .add_property(name, value)
    .to_str()

This interface is implemented by the concrete classes JsonSerializer and XmlSerializer.

The original example used a SongSerializer class. 
For the new application, you will implement something more generic, like ObjectSerializer:
"""

In [ ]:
#第一个block中，定义两个类，第二个类中用if elif else来区分，可读性较差
#第二个block中，定义两个类，第二个类中把ifelifelse集成在一个模块中，选择return各自的方法。
#第三个block中，进一步改善，定义了prodect component等。~                                   ~~推荐
#第四个block中，把无关的方法定义成类之外的函数

#第五个block，factory method